# Funções para cálculos de Computação Gráfica

## Quartenions:

### Detalhes dos formatos esperados dos argumentos:

In [29]:
#Quartenion linear:       np.array([w, i, j, k])
#Quartenion ângulo vetor: (norm, theta, np.array([x, y, z]))

In [30]:
import numpy as np
import math
from functools import reduce

In [31]:
def para_graus(theta):
    return theta / math.pi * 180


def para_radiano(theta):
    return theta / 180 * math.pi


cosd = lambda theta: math.cos(para_radiano(theta))


sind = lambda theta: math.sin(para_radiano(theta))


tand = lambda theta: math.tan(para_radiano(theta))


cotand = lambda theta: 1 / tand(theta)


def normaliza(v):
    norm = np.linalg.norm(v)
    
    if norm == 0: 
        return v
    
    return v / norm

In [32]:
def para_angulo_vetor(quartenion):
    norm = np.linalg.norm(quartenion)
    normalizado = normaliza(quartenion)
    
    theta = 2 * math.acos(normalizado[0])
    v = [normalizado[i] / math.sin(theta / 2) for i in range(1, 4)]
    
    q = np.array(v)
    
    return (norm, para_graus(theta), q)

In [33]:
def para_literal(quartenion):
    norm, theta, v = quartenion
    
    if np.linalg.norm(v) != 1:
        v = v / np.linalg.norm(v)
    
    theta = para_radiano(theta)
    
    w = math.cos(theta / 2)
    v = [v[i] * math.sin(theta / 2) for i in range(0, 3)]
    
    return norm * np.array([w] + v)

In [34]:
# recebe um quartenion e te devolve ele garantidamente em formato literal
def garante_literal(quartenion):
    if type(quartenion) != tuple:
        return quartenion
    
    return para_literal(quartenion)

In [35]:
# Soma dois quartenions em qualquer formato e retorna a soma deles no formato de q1
def soma(q1, q2):
    q1_literal = garante_literal(q1)
    q2_literal = garante_literal(q2)
        
    q3 = np.array([q1_literal[i] + q2_literal[i] for i in range(0, 4)])
    
    if type(q1) == tuple:
        return literal_para_angulo_vetor(q3)
    
    return q3

In [36]:
# Multiplica dois quartenions em qualquer formato e retorna a soma deles no formato de q1
def multiplica(q1, q2):
    q1_l = garante_literal(q1)
    q2_l = garante_literal(q2)
    
    w1 = q1_l[0]
    v1 = q1_l[1:]
    
    w2 = q2_l[0]
    v2 = q2_l[1:]
    
    w = w1 * w2 - np.dot(v1, v2)
    v = w1*v2 + w2*v1 + np.cross(v1, v2)
    
    q3 = np.insert(v, 0, w)
    
    if type(q1) == tuple:
        return literal_para_angulo_vetor(q3)
    
    return q3

In [37]:
def modulo(quartenion):
    q_l = garante_literal(quartenion)
    return np.linalg.norm(q_l)

In [38]:
# Devolve o quartenion conjugado na forma de entrada
def conjugado(quartenion):
    q_l = garante_literal(quartenion)
    q_ll = np.append(q_l[0], -1 * q_l[1:])
    
    if type(quartenion) == tuple:
        return para_angulo_vetor(q_ll)
    
    return q_ll

In [39]:
# Rotaciona um ponto usando um quartenion e devolve o ponto rotacionado
def rotaciona(ponto, quartenion):
    pq = np.array([0] + ponto)
    q_l = normaliza(garante_literal(quartenion))
    q_ll = conjugado(q_l)
    pql = multiplica(multiplica(q_l, pq), q_ll)
    
    return pql[1:]

In [40]:
q1 = np.array([3., 1, -2, -2])
q2 = np.array([-2., 0, 1, 2])

para_literal(para_angulo_vetor(q1))

multiplica(q1, q2)

modulo(np.array([-2, 0, 1, 2]))

para_literal((1, 85, np.array([3., -2, -1])))

rotaciona([7, -1, 4], (1, 85, np.array([3, -2, -1])))

para_angulo_vetor(np.array([0.8805, 0.2798, -0.3647, -0.1159]))

(1.000003594993538,
 56.595291053521116,
 array([ 0.59022845, -0.76932207, -0.24448705]))

## Transformações básicas e câmera virtual

In [41]:
# As funções abaixo retornam matrizes que realizam a dita transformação em um ponto.
# A de rotação geral está definida acima e rotaciona diretamente o ponto
def translacao(tx, ty, tz):
    trans = np.identity(4)
    trans[0:, 3] = [tx, ty, tz, 1]
    
    return trans


def escala(sx, sy, sz):
    scale = np.identity(4)
    scale[0, 0] = sx
    scale[1, 1] = sy
    scale[2, 2] = sz
    
    return scale


def rotacao_x(theta):
    return np.array([[1, 0          , 0           , 0],
                     [0, cosd(theta), -sind(theta), 0],
                     [0, sind(theta), cosd(theta) , 0],
                     [0, 0          , 0           , 1]])


def rotacao_y(theta):
    return np.array([[cosd(theta) , 0, sind(theta), 0],
                     [0           , 1, 0          , 0],
                     [-sind(theta), 0, cosd(theta), 0],
                     [0           , 0, 0          , 1]])


def rotacao_z(theta):
    return np.array([[cosd(theta), -sind(theta), 0, 0],
                     [sind(theta), cosd(theta) , 0 ,0],
                     [0          , 0           , 1, 0],
                     [0          , 0           , 0, 1]])


# Recebe uma série de transformações e retorna uma equivalente. 
# Elas devem ser passadas na ordem contraria que serão aplicadas
def chain(*transformacoes):
    return reduce(np.dot, transformacoes)

In [42]:
chain(translacao(2, 3, 4), rotacao_y(45), escala(3, 3, 3))

array([[ 2.12132034,  0.        ,  2.12132034,  2.        ],
       [ 0.        ,  3.        ,  0.        ,  3.        ],
       [-2.12132034,  0.        ,  2.12132034,  4.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [93]:
#Funções usadas para ajudar a transformar o SG em Sistema de Janela
def matriz_viewport(x0, y0, x1, y1):
    width = x1 - x0
    height = y1 - y0
    
    T = translacao(x0 + width/2, y0 + height/2, 1/2)
    S = escala(width/2, height/2, 1/2)
    
    return np.dot(T, S)


def matriz_view(p_camera, UP, foco):
    P = np.array(p_camera)
    F = np.array(foco)
    UP = np.array(UP)
    
    n = -normaliza(F - P)
    u = normaliza(np.cross(UP, n))
    v = np.cross(n, u)
    
    R = np.identity(4)
    R[0:3, 0] = u[0:3]
    R[0:3, 1] = v[0:3]
    R[0:3, 2] = n[0:3]
    
    T = translacao(p_camera[0], p_camera[1], p_camera[2])
    
    return np.linalg.inv(np.dot(T, R))


# left, right, bottom, top, near, far
def matriz_ortografica(l, r, b, t, n, f):
    S = escala(2/(r-l), 2/(t-b), -2/(f-n))
    T = translacao(-l - (r-l)/2, -b - (t-b)/2, -n - (f-n)/2)
    
    return np.dot(S, T)


def matriz_perspectiva(fov, aspecto, near, far):
    return np.array([[cotand(fov/2)/aspecto, 0            , 0                     , 0                     ],
                     [0                    , cotand(fov/2), 0                     , 0                     ],
                     [0                    , 0            , -(far+near)/(far-near), -2*far*near/(far-near)],
                     [0                    , 0            , -1                    , 0                     ]])


# Ph    : ponto homogêneo
# P3d   : omogêneo -> 3D
# Ppixel: coordenada no pixel (no dispositivo)
def calcula_pixel(ponto, m_projecao, m_view, m_model, m_viewport):
    Ph = chain(m_projecao, m_view, m_model, ponto)
    H = Ph[3]
    P3d = Ph / H
    Ppixel = np.dot(m_viewport, P3d)
    
    return [Ph, P3d, Ppixel]

In [44]:
m_v = matriz_view([0, 0, 0], [0, 1, 0], [1, 1, 1])
m_vp = matriz_viewport(0, 0, 800, 600)
m_p = matriz_ortografica(-2, 2, -2, 2, 1, 5)

print(m_v)
print(m_vp)
print(m_p)

calcula_pixel([1, 1, 0, 1], m_p, m_v, chain(translacao(1, 1, 1), rotacao_z(90)), m_vp)[2]

[[-7.07106781e-01 -7.85046229e-17  7.07106781e-01 -0.00000000e+00]
 [-4.08248290e-01  8.16496581e-01 -4.08248290e-01  0.00000000e+00]
 [-5.77350269e-01 -5.77350269e-01 -5.77350269e-01 -0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[400.    0.    0.  400. ]
 [  0.  300.    0.  300. ]
 [  0.    0.    0.5   0.5]
 [  0.    0.    0.    1. ]]
[[ 0.5  0.   0.   0. ]
 [ 0.   0.5  0.   0. ]
 [ 0.   0.  -0.5  1.5]
 [ 0.   0.   0.   1. ]]


array([541.42135624, 483.71173071,   1.6830127 ,   1.        ])

In [45]:
matriz_perspectiva(60, 4/3, 1, 5)

array([[ 1.29903811,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  1.73205081,  0.        ,  0.        ],
       [ 0.        ,  0.        , -1.5       , -2.5       ],
       [ 0.        ,  0.        , -1.        ,  0.        ]])

In [46]:
np.dot(matriz_viewport(20, 35, 520, 335), np.array([0, 0.3, 0.9, 1]))

array([270.  , 230.  ,   0.95,   1.  ])

In [47]:
np.dot(matriz_ortografica(-10, 10, -10, 10, 1, 10), np.array([5, -9, -6, 1]))

array([ 0.5       , -0.9       ,  2.55555556,  1.        ])

In [48]:
np.dot(matriz_view([10, 2, 3], [0, 1, 1], [0, 1, -3]), np.array([-5, 8, -4, 1]))

array([  3.66666667,   6.63551114, -15.89105243,   1.        ])

In [49]:
proj = matriz_ortografica(-5, 15, -5, 10, 1, 20)
view = matriz_view([0, 0, 0], [0, 1, 0], [1, 1, 1])

chain(proj, view, translacao(-3, 2, -6), rotacao_x(50), np.array([1, 0, 1, 1]))

array([-0.73739076,  0.20147865,  0.73313011,  1.        ])

In [50]:
np.dot(matriz_perspectiva(60, 4/3, 0.1, 20), np.array([2, -1, -5, 1]))

array([ 2.59807621, -1.73205081,  4.84924623,  5.        ])

In [51]:
view = matriz_view([0, 0, 0], [0, 1, 0], [1, 1, 1])
proj = matriz_perspectiva(45, 16/9, 0.1, 10)
viewport = matriz_viewport(0, 0, 1920, 1080)

calcula_pixel([4, 4, 5, 1], proj, view, chain(translacao(-3, 2, -6), rotacao_x(50)), viewport)[2]

array([389.13054573, 633.02213208,   0.9234488 ,   1.        ])

## Sombreamento e iluminação

In [96]:
def calcula_atenuacao(luz, kc, kl, kd, d):
    fator_atenuacao = 1 / (kc + kl*d + kd*d*d)
    return np.array(luz) * fator_atenuacao


def calcula_angulo_spotlight(ponto_luz, direcao_luz, ponto):
    v_luz = normaliza(direcao_luz)
    l = normaliza(np.array(ponto) - np.array(ponto_luz))

    return para_graus(math.acos(np.dot(l, v_luz)))


def iluminacao_difusa(luz, ponto_luz, ponto, normal):
    normal = normaliza(normal)
    l = normaliza(np.array(ponto_luz) - np.array(ponto))
    
    return np.array(luz) * max(0, np.dot(l, normal))


def cor_difusa(luz, ponto_luz, ponto, normal, Kd):
    Id = iluminacao_difusa(luz, ponto_luz, ponto, normal)
    return Id * Kd


def cor_ambiente(luz, Ka):
    return np.array(luz) * Ka


def iluminacao_phong(luz, ponto_luz, ponto, normal, sh, ponto_camera):
    l = normaliza(np.array(ponto_luz) - np.array(ponto))
    n = normaliza(normal)
    ln = np.dot(l, n)
    
    if (ln <= 0):
        return np.zeros(3)
    
    v = normaliza(np.array(ponto_camera) - np.array(ponto))
    r = 2 * np.dot(l, n) * n - l
    Is = np.array(luz) * math.pow(max(0, np.dot(v, r)), sh)
    return Is


def cor_phong(luz, ponto_luz, ponto, normal, Ks, sh, ponto_camera):
    Is = iluminacao_phong(luz, ponto_luz, ponto, normal, sh, ponto_camera)
    return Ks * Is


def iluminacao_bling_phong(luz, ponto_luz, ponto, normal, sh, ponto_camera):
    l = normaliza(np.array(ponto_luz) - np.array(ponto))
    n = normaliza(normal)
    ln = np.dot(l, n)
    
    if (ln <= 0):
        return np.zeros(3)
    
    v = normaliza(np.array(ponto_camera) - np.array(ponto))
    h = normaliza(l + v)
    Is = np.array(luz) * math.pow(max(0, np.dot(h, n)), sh)
    return Is

In [98]:
print("1-", calcula_atenuacao([0.5, 0.9, 0.3], 1, 1.3, 1.04, 2))

print("2-", calcula_angulo_spotlight([0, 0, 0], [1, 1, 1], [4, 2, 0]))

print("3-", iluminacao_difusa([0, 1, 1], [1, 2, 2], [0, 1, 1], [0, 0, 1]))

print("4-", cor_difusa([0, 1, 1], [1, 2, 2], [0, 1, 1], [0, 0, 1], [1, 0.5, 0.8]))

print("5-", cor_ambiente([0.1, 0.2, 0.2], [1, 0.5, 0.8]))

print("6-", iluminacao_phong([0.8, 0, 0.5], [1, 2, 2], [0, 1, 1], [0, 0, 1], 2, [1, 0, 2]))

print("7-", cor_phong([0.8, 0, 0.5], [1, 2, 2], [0, 1, 1], [0, 0, 1], [0.7, 0.7, 0.7], 2, [1, 0, 2]))

print("8-", iluminacao_phong([0.8, 0, 0.5], [1, 2, 2], [0, 1, 1], [0, 0, 1], 2, [1, 0, 2]))

1- [0.06443299 0.11597938 0.03865979]
2- 39.23152048359225
3- [0.         0.57735027 0.57735027]
4- [0.         0.28867513 0.46188022]
5- [0.1  0.1  0.16]
6- [0.08888889 0.         0.05555556]
7- [0.06222222 0.         0.03888889]
8- [0.08888889 0.         0.05555556]


In [70]:
ia = calcula_atenuacao([.1, .1, .1], 1, .14, .07, np.linalg.norm([2, -1, -1]))
_id = calcula_atenuacao([.6, .7, .8], 1, .14, .07, np.linalg.norm([2, -1, -1]))

contrib_ambiente = cor_ambiente(ia, [.6, .6, .6])
print(contrib_ambiente)

contrib_difusa = cor_difusa(_id, [6, 0, -2], [4, 1, -1], [1, 0, 0], [.6, .6, .6])
print(contrib_difusa)

contrib_ambiente + contrib_difusa

[0.03403428 0.03403428 0.03403428]
[0.16673323 0.1945221  0.22231097]


array([0.2007675 , 0.22855638, 0.25634525])

In [76]:
mvp = matriz_viewport(10, 40, 610, 440)
np.dot(mvp, np.array([.95, .7, -0.5, 1]))

array([5.95e+02, 3.80e+02, 2.50e-01, 1.00e+00])

In [80]:
mv = matriz_view([1, 9, -2], [0, 1, 0], [-1, -2, 2])
np.dot(mv, np.array([6, -1, -4, 1]))

array([-3.57770876, -7.49477359, -7.74779767,  1.        ])

In [83]:
np.array([.5, .5, .5]) * iluminacao_bling_phong([.9, .9, .9], [5, 4, -1], [-1, 2, 5], [0, 1, 0], 3, [-3, 4, 7])

array([0.4256459, 0.4256459, 0.4256459])

In [94]:
mo = matriz_ortografica(-3, 3, -3, 3, .1, 10)
np.dot(mo, np.array([3, 1, -3, 1]))

array([1.        , 0.33333333, 1.62626263, 1.        ])

In [90]:
mp = matriz_perspectiva(60, 1.5, .1, 10)
ponto_parcial = np.dot(mp, np.array([3, 1, -3, 1]))
ponto_parcial / ponto_parcial[3]

array([1.15470054, 0.57735027, 0.95286195, 1.        ])

In [95]:
np.array([.5, .5, .5]) * iluminacao_phong([.9, .9, .9], [5, 4, -1], [-1, 2, 5], [0, 1, 0], 3, [-3, 4, 7])

array([0.35866942, 0.35866942, 0.35866942])